In [58]:
import os 
import sys
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from astropy.table import Table
from astroML.filters import savitzky_golay, wiener_filter
%matplotlib inline

In [59]:
## automatically reload any modules read below that might have changed (e.g. plots)
%load_ext autoreload
%autoreload 2
# importing ZI tools for thermal analysis: 
import sputnikTools as s 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
## specifications for Perun from Matija Makoter based on CAD model (April 2021)
def getInputParamsPerun(Perun="", case=0):   
    
    if (Perun==""):
        Perun = s.getInputParamsDefault()

    # area-weighted surface properties for Perun
    if case==0:
        # original values from Matija Makoter
        alphaPerun, epsPerun, areaPerun = getPerunSurfaceParamsDefault()
    if case==1:
        # change Al frame properties to copper properties 
        alphaPerun, epsPerun, areaPerun = getPerunSurfaceParamsCopper()

    # surface emissivity
    Perun['alphaS'] = alphaPerun  
    Perun['epsT'] = epsPerun      
    Perun['alphaIR'] = Perun['epsT']  # usual ansatz
    
    # thermal inertia
    Perun['mass'] = 2.684    # kg 
    Perun['C'] = 721.0       # J/kg/K, mass-averaged value
    # battery charging/power dissipation
    Perun['etaCell'] = 0.2 # fraction of energy for charging
    
    # effective areas for absorption
    Perun['etaS'] = 0.21 
    Perun['etaE'] = 0.36
    
    ## orbit
    Perun['h'] = 550.0     # in km, orbit's altitude
    Perun['PorbMin'] = 96  # orbital period in minutes
    Perun['etaP'] = 0.375# fraction of orbit in eclipse: 36/96 
    
    return Perun

def getPerunSurfaceParamsDefault(verbose=False):
    ## optical absorptivity, IR emissivity, area (in m2) and C (in J/kg/K)     

    # solar panels
    alphaSolPan = 0.92
    epsSolPan = 0.85 
    AreaSolPan = 0.048772
    CSolPan = 324

    # ceramics
    alphaCer = 0.90
    epsCer = 0.90
    AreaCer = 0.022201
     
    # Al frame 
    alphaAlFrameR = 0.86 
    epsAlFrameR = 0.86 
    AreaAlFrameR = 0.024926 
    CAlFrameR = 768

    # copper
    alphaCu = 0.30
    epsCu = 0.03
    AreaCu = 0.003713
    
    # Kapton foil (S band antenna)
    alphaKap = 0.11 
    epsKap = 0.33
    AreaKap = 0.01066  

    # total area in square meters 
    areaPerun = AreaSolPan + AreaCer + AreaAlFrameR + AreaCu + AreaKap  
    alphaPerun = alphaSolPan*AreaSolPan + alphaCer*AreaCer + alphaAlFrameR*AreaAlFrameR
    alphaPerun = (alphaPerun + alphaCu*AreaCu + alphaKap*AreaKap)/areaPerun
    epsPerun = epsSolPan*AreaSolPan + epsCer*AreaCer + epsAlFrameR*AreaAlFrameR
    epsPerun = (epsPerun + epsCu*AreaCu + epsKap*AreaKap)/areaPerun

    return alphaPerun, epsPerun, areaPerun


def getPerunSurfaceParamsCopper(verbose=False):
    ## optical absorptivity, IR emissivity, area (in m2) and C (in J/kg/K)     

    # solar panels
    alphaSolPan = 0.92
    epsSolPan = 0.85 
    AreaSolPan = 0.048772
    CSolPan = 324

    # ceramics
    alphaCer = 0.90
    epsCer = 0.90
    AreaCer = 0.022201
     
    # Al frame 
    # to show Cu effects: 
    #alphaAlFrameR = 0.86 
    #epsAlFrameR = 0.86 
    alphaAlFrameR = 0.30
    epsAlFrameR = 0.03

    AreaAlFrameR = 0.024926 
    CAlFrameR = 768

    # copper
    alphaCu = 0.30
    epsCu = 0.03
    AreaCu = 0.003713
    
    # Kapton foil (S band antenna)
    alphaKap = 0.11 
    epsKap = 0.33
    AreaKap = 0.01066  

    # total area in square meters 
    areaPerun = AreaSolPan + AreaCer + AreaAlFrameR + AreaCu + AreaKap  
    alphaPerun = alphaSolPan*AreaSolPan + alphaCer*AreaCer + alphaAlFrameR*AreaAlFrameR
    alphaPerun = (alphaPerun + alphaCu*AreaCu + alphaKap*AreaKap)/areaPerun
    epsPerun = epsSolPan*AreaSolPan + epsCer*AreaCer + epsAlFrameR*AreaAlFrameR
    epsPerun = (epsPerun + epsCu*AreaCu + epsKap*AreaKap)/areaPerun

    return alphaPerun, epsPerun, areaPerun

In [66]:
# set default Perun parameters 
Perun = getInputParamsPerun()
coldPerun = s.getInputParamsCold(getInputParamsPerun())  
hotPerun = s.getInputParamsHot(getInputParamsPerun())  
alphaPerun = Perun['alphaS']
epsPerun = Perun['epsT']
print(Perun['alphaS'], Perun['epsT'])  # 0.8032320081253627 0.7844480013058618

0.6766490133488102 0.5968339197620429


In [45]:
## hot and cold cases for Perun
t1 = 'Perun hot case (beta=90 orbit)'  
t2 = 'Perun cold case (beta=0 orbit)'  

# default randomized orientations
hotPerun['etaS'] = 0.21 
hotPerun['etaE'] = 0.36
coldPerun['etaS'] = 0.21 
coldPerun['etaE'] = 0.36

verbose = True
time, Temps1 = s.doOneCase(t1, alphaPerun, epsPerun, hotPerun, verbose)  
time, Temps2 = s.doOneCase(t2, alphaPerun, epsPerun, coldPerun, verbose)  

name = 'Perun-hotVScold'
s.TempsPlot(time, Temps2, t2, Temps1, t1, "", "", name)


Perun hot case (beta=90 orbit) Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.986114226639582 0.731711549342613 6.221821126072711 4.943565155196439
Qsun= 30.939646902054907  Qeclipse= 11.16538628126915
Hot Eq. Temp: 288.79K = 15.64℃
Cold Eq. Temp: 223.83K = -49.32℃
adjusting Tmax= 288.94067938195263 to: Thot= 288.79275170283637
doOneCase: temp. range = 288.28303528629607 288.50395895113354
     in Celsius: range = 15.133035286296092 15.353958951133563

Perun cold case (beta=0 orbit) Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 22.29932700957632 5.02093006415494 5.264617875907678 3.4150321342164074
Qsun= 30.535855669109093  Qeclipse= 8.679650010124085
Hot Eq. Temp: 287.85K = 14.70℃
Cold Eq. Temp: 210.18K = -62.97℃
doOneCase: temp. range = 258.39266406307144 273.82201491081094
     in Celsius: range = -14.757335936928541 0.6720149108109581
pero2 96.0 96.0


In [46]:
## hot and cold cases for Perun - extreme cases 
t1 = 'Perun hot case, extreme A'  
t2 = 'Perun cold case, extreme A'  

hotPerun['etaS'] = 0.31 
hotPerun['etaE'] = 0.38
coldPerun['etaS'] = 0.10
coldPerun['etaE'] = 0.34

verbose = True
time, Temps1 = s.doOneCase(t1, alphaPerun, epsPerun, hotPerun, verbose)  
time, Temps2 = s.doOneCase(t2, alphaPerun, epsPerun, coldPerun, verbose)  

name = 'Perun-hotVScoldExtremeA'
s.TempsPlot(time, Temps2, t2, Temps1, t1, "", "", name)


Perun hot case, extreme A Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 35.40807338218224 0.772362190972758 6.567477855298972 7.236087114630999
Qsun= 42.74791342845397  Qeclipse= 13.803564969929973
Hot Eq. Temp: 313.10K = 39.95℃
Cold Eq. Temp: 236.02K = -37.13℃
adjusting Tmax= 313.1387002052795 to: Thot= 313.1022444907002
doOneCase: temp. range = 312.35975890037133 312.78914224620945
     in Celsius: range = 39.209758900371355 39.639142246209474

Perun cold case, extreme A Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 10.618727147417296 4.741989505035221 4.972139105023919 1.9200895815565646
Qsun= 19.1808020085425  Qeclipse= 6.892228686580483
Hot Eq. Temp: 256.26K = -16.89℃
Cold Eq. Temp: 198.40K = -74.75℃
doOneCase: temp. range = 235.13620521861282 243.86043524106165
     in Celsius: range = -38.01379478138716 -29.289564758938326
pero2 96.0 96.0


In [14]:
## extra heat on
t1 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 5.0 
label = 'Extra 5 W when T$<$273 K'
name = 'SOCi-cold-heated5'

# analytic
timeA, TempsA = s.doOneCase(t1, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t1, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))



## extra heat on
t2 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 2.0 
label = 'Extra 2 W when T$<$273 K'
name = 'SOCi-cold-heated2'

# analytic
timeA, TempsA = s.doOneCase(t2, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t2, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))


## extra heat on
t3 = 'SOC-i cold case'  
coldSOCi['etaS'] = 0.21 
coldSOCi['etaE'] = 0.36
Tcritical = 273.15
Pcritical = 10.0 
label = 'Extra 10 W when T$<$273 K'
name = 'SOCi-cold-heated10'

# analytic
timeA, TempsA = s.doOneCase(t3, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t3, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))



t4 = 'SOC-i cold case, extreme A'  
coldSOCi['etaS'] = 0.10
coldSOCi['etaE'] = 0.34
Tcritical = 273.15
Pcritical = 5.0 
label = 'Extra 5 W when T$<$273 K'
name = 'SOCi-cold-extrA-heated5'

# analytic
timeA, TempsA = s.doOneCase(t4, alphaSOCi, epsSOCi, coldSOCi, verbose)   
# numerical test 
timeN, TempsN = s.getNumSolution(coldSOCi, 1.01*TempsA[0], Tcritical, Pcritical)
# plot
s.TempsPlotCompare(timeA, TempsA, timeN, TempsN, t4, label, name) 
print('Active control range:', np.min(TempsN-273.15), np.max(TempsN-273.15))


SOC-i cold case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.056341000000007 5.191379795748719 5.319730560305015 3.530965099468591
Qsun= 31.44887229637259  Qeclipse= 8.850695659773606
Hot Eq. Temp: 289.22K = 16.07℃
Cold Eq. Temp: 210.65K = -62.50℃
doOneCase: temp. range = 258.98122604261715 274.44572157726554
     in Celsius: range = -14.168773957382825 1.2957215772655672
Active control range: -7.029643321464789 6.401408390095526

SOC-i cold case Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 23.056341000000007 5.191379795748719 5.319730560305015 3.530965099468591
Qsun= 31.44887229637259  Qeclipse= 8.850695659773606
Hot Eq. Temp: 289.22K = 16.07℃
Cold Eq. Temp: 210.65K = -62.50℃
doOneCase: temp. range = 258.98122604261715 274.44572157726554
     in Celsius: range = -14.168773957382825 1.2957215772655672
Active control range: -10.073203396790575 4.47740863232

In [47]:
coldPerun

{'Fsun': 1322,
 'FIR': 220,
 'rhoE': 0.25,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.375,
 'falb': 0.62,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.1,
 'etaE': 0.34,
 'alphaS': 0.8032320081253627,
 'epsT': 0.7844480013058618,
 'alphaIR': 0.7844480013058618,
 'mass': 2.684,
 'C': 721.0,
 'etaCell': 0.2}

In [48]:
Perun

{'Fsun': 1372,
 'FIR': 240,
 'rhoE': 0.3,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.375,
 'falb': 0.62,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.21,
 'etaE': 0.36,
 'alphaS': 0.8032320081253627,
 'epsT': 0.7844480013058618,
 'alphaIR': 0.7844480013058618,
 'mass': 2.684,
 'C': 721.0,
 'etaCell': 0.2}

In [49]:
hotPerun

{'Fsun': 1422,
 'FIR': 260,
 'rhoE': 0.35,
 'h': 550.0,
 'PorbMin': 96,
 'etaP': 0.0,
 'falb': 0.06,
 'fE': 0.8473786384498566,
 'Atot': 0.1,
 'etaS': 0.31,
 'etaE': 0.38,
 'alphaS': 0.8032320081253627,
 'epsT': 0.7844480013058618,
 'alphaIR': 0.7844480013058618,
 'mass': 2.684,
 'C': 721.0,
 'etaCell': 0.2}

In [50]:
hotPerun['Atot'] = 1.0
hotPerun['etaS'] = 1.0
hotPerun['etaE'] = 1.0
hotPerun['etaCell'] = 0.0
Qsun2, Qref2, QIR2, Qdissip2 = s.getAllHeatQ(hotPerun)
print(Qsun2, Qref2, QIR2, Qdissip2)
QinSun, QinEclipse = s.getQin(hotPerun)
QinSun, QinEclipse

1142.1959155542656 20.32532081507258 172.82836461313087 0.0


(1335.349600982469, 172.82836461313087)

In [51]:
coldPerun['Atot'] = 1.0
coldPerun['etaS'] = 1.0
coldPerun['etaE'] = 1.0
coldPerun['etaCell'] = 0.0
Qsun2, Qref2, QIR2, Qdissip2 = s.getAllHeatQ(coldPerun)
print(Qsun2, Qref2, QIR2, Qdissip2)
QinSun, QinEclipse = s.getQin(coldPerun)
QinSun, QinEclipse

1061.8727147417294 139.47027955985945 146.23938544187996 0.0


(1347.5823797434687, 146.23938544187996)

In [52]:
1142        /1061      

1.0763430725730443

In [53]:
20.3        /    139.5 

0.14551971326164875

In [70]:
### show the effect of placing copper foil on exposed Al frame 
Perun0 = getInputParamsPerun(Perun="", case = 0)
coldPerun = s.getInputParamsCold(getInputParamsPerun())  
alphaPerun0 = Perun0['alphaS']
epsPerun0 = Perun0['epsT']
PerunCu = getInputParamsPerun(Perun="", case = 1)
alphaPerunCu = PerunCu['alphaS']
epsPerunCu = PerunCu['epsT']
print(alphaPerun0, epsPerun0, alphaPerunCu, epsPerunCu) 

0.8032320081253627 0.7844480013058618 0.6766490133488102 0.5968339197620429


In [72]:
## hot and cold cases for Perun
t1 = 'Perun cold case, default'  
t2 = 'Perun cold case, Cu over Al'  

# default randomized orientations
coldPerun['etaS'] = 0.21 
coldPerun['etaE'] = 0.36

verbose = True
time, Temps1 = s.doOneCase(t2, alphaPerun0, epsPerun0, coldPerun, verbose)  
time, Temps2 = s.doOneCase(t2, alphaPerunCu, epsPerunCu, coldPerun, verbose)  

name = 'Perun-hotVScold-Cu'
s.TempsPlot(time, Temps1, t1, Temps2, t2, "", "", name)


Perun cold case, Cu over Al Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 22.29932700957632 5.02093006415494 5.264617875907678 3.4150321342164074
Qsun= 30.535855669109093  Qeclipse= 8.679650010124085
Hot Eq. Temp: 287.85K = 14.70℃
Cold Eq. Temp: 210.18K = -62.97℃
doOneCase: temp. range = 258.39266406307144 273.82201491081094
     in Celsius: range = -14.757335936928541 0.6720149108109581

Perun cold case, Cu over Al Sphere, Temperature Extremes
-------------------------------------------------------
Q (sun, alb, IR, diss): 18.78512990858967 4.229671302483225 4.0054949693245145 2.876850151384112
Qsun= 25.294186089566942  Qeclipse= 6.882345120708626
Hot Eq. Temp: 294.03K = 20.88℃
Cold Eq. Temp: 212.36K = -60.79℃
doOneCase: temp. range = 264.01148992881076 277.08824833788464
     in Celsius: range = -9.138510071189216 3.9382483378846587
pero2 96.0 96.0
